<a href="https://colab.research.google.com/github/shreyshrey/ML/blob/main/OCOMP5203M_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OCOM5203M Assignment 1 - Image Caption Generation [100 marks]

The maximum number of marks for each part are shown in the section headers. As indicated in the main heading above, the overall assessment carries a maximum of 100 marks.

This summative assessment is weighted 50% of the final grade for the module.

### Motivation 

Through this coursework, you will:

> 1. Understand the principles of text pre-processing and vocabulary building.
> 2. Gain experience working with an image to text model.
> 3. Use and compare two different text similarity metrics for evaluating an image to text model, and understand evaluation challenges.


### Setup and resources 

Having a GPU will speed up the image feature extraction process. If you would like to use a GPU, please refer to the module website for recommended working environments with GPUs.

Please implement the coursework using Python and PyTorch, and refer to the notebooks and exercises provided.

This coursework will use a subset of the [COCO "Common Objects in Context" dataset](https://cocodataset.org/) for image caption generation. COCO contains 330K images, of 80 object categories, and at least five textual reference captions per image. Our subset consists of 5029 of these images, each of which has five or more different descriptions of the salient entities and activities, and we will refer to it as COCO_5029.

To download the data:

> 1. **Images**: download the zip file "coco_subset_images.zip (812MB)" [here](https://leeds365-my.sharepoint.com/:f:/g/personal/busmnom_leeds_ac_uk/EuAH3b6a4g9IjTNhroLLXPoB6ho6cwxYSNh885ZzrktYZA?e=QGSYpf).
> 2. **Reference captions**: on the COCO [download page](https://cocodataset.org/#download), download the file named "2017 Train/Val annotations (241MB)". 
> 3. **Image meta data**: as our set is a subset of full COCO dataset, we have created a CSV file containing relevant meta data for our particular subset of images. You can download it also from Drive, "coco_subset_meta.csv" at the same link as 1.


### Submission

Please submit the following:

> 1. Your completed Jupyter notebook file, in .ipynb format. **Do not change the file name or the automatic grading will be affected.**
> 2. The .html version of your notebook; File > Download as > HTML (.html). Check that all cells have been run and all outputs (including all graphs you would like to be marked) displayed in the .html for marking.


Final note:

> **Please include in this notebook everything that you would like to be marked, including figures. Under each section, put the relevant code containing your solution. You may re-use functions you defined previously, but any new code must be in the relevant section.** Feel free to add as many code cells as you need under each section.

Your student username (for example, ```sc15jb```):

Your full name:

## Imports

Feel free to add to this section as needed.

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os
import numpy as np

Detect which device (CPU/GPU) to use.

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


The basic principle of our image-to-text model is as pictured in the diagram below, where an Encoder network encodes the input image as a feature vector by providing the outputs of the last fully-connected layer of a pre-trained CNN (we use [ResNet-152](https://arxiv.org/abs/1512.03385)). This pretrained network has been trained on the complete ImageNet dataset and is thus able to recognise common objects. 

These features are then fed into a Decoder network along with the reference captions. As the image feature dimensions are large and sparse, the Decoder network includes a linear layer which downsizes them, followed by a batch normalisation layer to speed up training. Those resulting features, as well as the reference text captions, are passed into a recurrent network (we will use an RNN). 

The reference captions used to compute loss are represented as numerical vectors via an embedding layer whose weights are learned during training.

![Encoder Decoder](encoder_decoder_diagramv2022.png)

The Encoder-Decoder network could be coupled and trained end-to-end, without saving features to disk; however, this requires iterating through the entire image training set during training. We can make the training more efficient by decoupling the networks. 

We will first extract the feature representations of the images from the Encoder and save them (Part 1). During training of the Decoder (Part 3), we only need to iterate over the image feature data and the reference captions.

### Overview

> 1. Extracting image features 
> 2. Text preparation of training and validation data 
> 3. Training the decoder
> 4. Generating predictions on test data
> 5. Caption evaluation via BLEU score
> 6. Caption evaluation via Cosine similarity
> 7. Comparing BLEU and Cosine similarity



## 1 Extracting image features [11 marks]

### 1.1 EncoderCNN

Read through the template EncoderCNN class below and complete the class.

In [4]:
class EncoderCNN(nn.Module):
    def __init__(self):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(EncoderCNN, self).__init__()
        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]      # delete the last fc layer.
        # print(modules)

        # TO COMPLETE
        # keep all layers of the pretrained net except the last one

        
    def forward(self, images):
        """Extract feature vectors from input images."""
        with torch.no_grad():
          features = self.resnet(images)
        features = features.reshape(features.size(0), -1)
        features = self.bn(self.linear(features))
        return features
        # TO COMPLETE
        # remember no gradients are needed
        # return features 

In [5]:
# instantiate encoder and put into evaluation mode.
encoder = EncoderCNN().to(device)
encoder.eval()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

EncoderCNN()

### 1.2 Processing the images

Pass the images through the ```Encoder``` model, saving the resulting features for each image. You may like to use a ```Dataset``` and ```DataLoader``` to load the data in batches for faster processing, or you may choose to simply read in one image at a time from disk without any loaders.

Note that as this is a forward pass only, no gradients are needed. You will need to be able to match each image ID (the image name without file extension) with its features later, so we suggest either saving a dictionary of image ID: image features, or keeping a separate ordered list of image IDs.

Use this ImageNet transform provided.

In [ ]:
data_transform = transforms.Compose([ 
    transforms.ToTensor(),
    transforms.Resize(224), 
    transforms.CenterCrop(224), 
    transforms.Normalize((0.485, 0.456, 0.406),   # using ImageNet norms
                         (0.229, 0.224, 0.225))])

In [ ]:
# torch.save(features, 'features.pt')

## 2 Text preparation [23 marks]


### 2.1 Build the caption dataset

All our selected COCO_5029 images are from the official 2017 train set.

The ```coco_subset_meta.csv``` file includes the image filenames and unique IDs of all the images in our subset. The ```id``` column corresponds to each unique image ID.

The COCO dataset includes many different types of annotations: bounding boxes, keypoints, reference captions, and more. We are interested in the captioning labels. Open ```captions_train2017.json``` from the zip file downloaded from the COCO website. You are welcome to come up with your own way of doing it, but we recommend using the ```json``` package to initially inspect the data, then the ```pandas``` package to look at the annotations (if you read in the file as ```data```, then you can access the annotations dictionary as ```data['annotations']```).

Use ```coco_subset_meta.csv``` to cross-reference with the annotations from ```captions_train2017.json``` to get all the reference captions for each image in COCO_5029.

For example, you may end up with data looking like this (this is a ```pandas``` DataFrame, but it could also be several lists, or some other data structure/s):

<img src="df_caption_set.png" alt="images matched to caption" width="700"/>

In [ ]:
#connecting to google drive to get the files

In [6]:
from google.colab import drive
drive.mount('/content/drive/')
annotation_path = '/content/drive/MyDrive/Leeds MSc/DL/annotations'

Mounted at /content/drive/


In [7]:
meta_file = pd.read_csv('/content/drive/MyDrive/Leeds MSc/DL/coco_subset_meta.csv')

In [8]:
df = meta_file[['file_name','id']]
df.rename(columns = {'id':'image_id'}, inplace = True)
df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,file_name,image_id
0,000000262145.jpg,262145
1,000000262146.jpg,262146
2,000000524291.jpg,524291
3,000000262148.jpg,262148
4,000000393223.jpg,393223


In [127]:
df[df['image_id'] == 203564]

,file_name,image_id


In [52]:
import json
  
# Opening JSON file
f = open('/content/drive/MyDrive/Leeds MSc/DL/annotations/captions_train2017.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

print(data.keys())


dict_keys(['info', 'licenses', 'images', 'annotations'])


In [53]:
ann_list = data['annotations']
ann_df = pd.DataFrame.from_records(ann_list)
ann_df.shape

(591753, 3)

In [34]:
ann_df[ann_df['image_id']== 203564]

,image_id,id,caption
0,203564,37,A bicycle replica with a clock as the front wh...
8,203564,181,The bike has a clock as a tire.
19,203564,478,A black metal bicycle with a clock inside the ...
237,203564,6637,A bicycle figurine in which the front wheel is...
246,203564,6802,A clock with the appearance of the wheel of a ...


In [54]:
image_list = data['images']
image_df = pd.DataFrame.from_records(image_list)

In [41]:
image_df.shape

(118287, 8)

In [55]:
final_df = pd.merge(ann_df, image_df, how='inner', left_on='image_id', right_on='id')


In [56]:
# Index(['image_id', 'id_x', 'caption', 'license', 'file_name', 'coco_url',
#        'height', 'width', 'date_captured', 'flickr_url', 'id_y'],
#       dtype='object')
final_df = final_df.drop(['license', 'coco_url','height','width','date_captured','flickr_url','id_y'], axis=1)

In [57]:
#save df to csv
final_df.to_csv('df_caption_set.csv')


In [14]:
# df3[df3['image_id']== 203564]

### 2.2 Clean the captions

Create a cleaned version of each caption. If using dataframes, we suggest saving the cleaned captions in a new column; otherwise, if you are storing your data in some other way, create data structures as needed. 

**A cleaned caption should be all lowercase, and consist of only alphabet characters.**

Print out 10 original captions next to their cleaned versions to facilitate marking.

In [58]:
final_df['clean_caption'] = final_df.loc[:, 'caption']

In [59]:
import numpy as np

final_df['clean_caption'] = final_df['clean_caption'].map(lambda x: x.lower() if isinstance(x,str) else x)
final_df.tail(20)

,image_id,id_x,caption,file_name,clean_caption
591733,410182,829245,a plate of various food on a table,000000410182.jpg,a plate of various food on a table
591734,410182,829400,A plate of sandwiches and some french fries.,000000410182.jpg,a plate of sandwiches and some french fries.
591735,410182,829537,Plate with fries and three sandwiches topped w...,000000410182.jpg,plate with fries and three sandwiches topped w...
591736,410182,829629,Slider sandwiches served with fries are displa...,000000410182.jpg,slider sandwiches served with fries are displa...
591737,410182,829658,A long plate hold some fries with some sliders...,000000410182.jpg,a long plate hold some fries with some sliders...
591738,383652,829299,A full fridge door sits open in an empty kitchen,000000383652.jpg,a full fridge door sits open in an empty kitchen
591739,383652,829505,A open refrigerator door in a small kitchen.,000000383652.jpg,a open refrigerator door in a small kitchen.
591740,383652,829529,A refeigerator door is left open to shine its ...,000000383652.jpg,a refeigerator door is left open to shine its ...
591741,383652,829574,a refrigerator stocked full and left open with...,000000383652.jpg,a refrigerator stocked full and left open with...
591742,383652,829633,The light from the open refrigerator fills the...,000000383652.jpg,the light from the open refrigerator fills the...


In [60]:
final_df['clean_caption'] = final_df['clean_caption'].str.replace(r'[^\w\s]+', '')

<ipython-input-60-1c48044e704d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['clean_caption'] = final_df['clean_caption'].str.replace(r'[^\w\s]+', '')


In [61]:
final_df.tail(10)

,image_id,id_x,caption,file_name,clean_caption
591743,133071,829358,"Bread, sour cream, and guacamole are arrayed o...",000000133071.jpg,bread sour cream and guacamole are arrayed on ...
591744,133071,829471,A white plate of food on a dining table.,000000133071.jpg,a white plate of food on a dining table
591745,133071,829655,a slice of bread is covered with a sour cream ...,000000133071.jpg,a slice of bread is covered with a sour cream ...
591746,133071,829693,White Plate with a lot of guacamole and an ext...,000000133071.jpg,white plate with a lot of guacamole and an ext...
591747,133071,829717,A dinner plate has a lemon wedge garnishment.,000000133071.jpg,a dinner plate has a lemon wedge garnishment
591748,180285,829395,A couple of women with some stuffed animals.,000000180285.jpg,a couple of women with some stuffed animals
591749,180285,829607,Fans pose with stuffed animals at an ice rink.,000000180285.jpg,fans pose with stuffed animals at an ice rink
591750,180285,829636,"Two women smiling together, one holds a stuffe...",000000180285.jpg,two women smiling together one holds a stuffed...
591751,180285,829653,Two women smile for the camea while posing iwt...,000000180285.jpg,two women smile for the camea while posing iwt...
591752,180285,829665,Two women sit and pose with stuffed animals.,000000180285.jpg,two women sit and pose with stuffed animals


In [47]:
import string
alphabet = string.ascii_letters+string.punctuation
final_df.clean_caption.str.strip(alphabet).astype(bool).any()


True

In [62]:
final_df

,image_id,id_x,caption,file_name,clean_caption
0,203564,37,A bicycle replica with a clock as the front wh...,000000203564.jpg,a bicycle replica with a clock as the front wheel
1,203564,181,The bike has a clock as a tire.,000000203564.jpg,the bike has a clock as a tire
2,203564,478,A black metal bicycle with a clock inside the ...,000000203564.jpg,a black metal bicycle with a clock inside the ...
3,203564,6637,A bicycle figurine in which the front wheel is...,000000203564.jpg,a bicycle figurine in which the front wheel is...
4,203564,6802,A clock with the appearance of the wheel of a ...,000000203564.jpg,a clock with the appearance of the wheel of a ...
...,...,...,...,...,...
591748,180285,829395,A couple of women with some stuffed animals.,000000180285.jpg,a couple of women with some stuffed animals
591749,180285,829607,Fans pose with stuffed animals at an ice rink.,000000180285.jpg,fans pose with stuffed animals at an ice rink
591750,180285,829636,"Two women smiling together, one holds a stuffe...",000000180285.jpg,two women smiling together one holds a stuffed...
591751,180285,829653,Two women smile for the camea while posing iwt...,000000180285.jpg,two women smile for the camea while posing iwt...


### 2.3  Split the data

Split the data 70/10/20% into train/validation/test sets. **Be sure that each unique image (and all corresponding captions) only appear in a single set.**

We provide the function below which, given a list of unique image IDs and a 3-split ratio, shuffles and returns  a split of the image IDs.

If using a dataframe, ```df['image_id'].unique()``` will return the list of unique image IDs.

In [63]:
import random
import math

def split_ids(image_id_list, train=.7, valid=0.1, test=0.2):
    """
    Args:
        image_id_list (int list): list of unique image ids
        train (float): train split size (between 0 - 1)
        valid (float): valid split size (between 0 - 1)
        test (float): test split size (between 0 - 1)
    """
    list_copy = image_id_list.copy()
    random.shuffle(list_copy)
    
    train_size = math.floor(len(list_copy) * train)
    valid_size = math.floor(len(list_copy) * valid)
    
    return list_copy[:train_size], list_copy[train_size:(train_size + valid_size)], list_copy[(train_size + valid_size):]

In [65]:
final_df['image_id'].unique()

array([203564, 322141,  16977, ..., 383652, 133071, 180285])

### 2.3 Building the vocabulary

The vocabulary consists of all the possible words which can be used - both as input into the model, and as output predictions, and we will build it using the cleaned words found in the reference captions from the training set. In the vocabulary each unique word is mapped to a unique integer (a Python ```dictionary``` object).

A ```Vocabulary``` object is provided for you below to use.

In [ ]:
class Vocabulary(object):
    """ Simple vocabulary wrapper which maps every unique word to an integer ID. """
    def __init__(self):
        # intially, set both the IDs and words to dictionaries with special tokens
        self.word2idx = {'<pad>': 0, '<unk>': 1, '<end>': 2}
        self.idx2word = {0: '<pad>', 1: '<unk>', 2: '<end>'}
        self.idx = 3

    def add_word(self, word):
        # if the word does not already exist in the dictionary, add it
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            # increment the ID for the next word
            self.idx += 1

    def __call__(self, word):
        # if we try to access a word not in the dictionary, return the id for <unk>
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)


Collect all words from the cleaned captions in the **training and validation sets**, ignoring any words which appear 3 times or less; this should leave you with roughly 2200 words (plus or minus is fine). As the vocabulary size affects the embedding layer dimensions, it is better not to add the very infrequently used words to the vocabulary.

Create an instance of the ```Vocabulary()``` object and add all your words to it.

### 2.4 The Dataset and DataLoader

Create a PyTorch ```Dataset``` class and a corresponding ```DataLoader``` for the inputs to the decoder. Create three sets: one each for training, validation, and test. Set ```shuffle=True``` for the training set DataLoader.

The ```Dataset``` function ```__getitem__(self, index)``` should return three Tensors:

>1. A Tensor of image features, dimension (1, 2048).
>2. A Tensor of integer word ids representing the reference caption; use your ```Vocabulary``` object to convert each word in the caption to a word ID. Be sure to add the word ID for the ```<end>``` token at the end of each caption, then fill in the the rest of the caption with the ```<pad>``` token so that each caption has uniform lenth (max sequence length) of **47**.
>3. A Tensor of integers representing the true lengths of every caption in the batch (include the ```<end>``` token in the count).


Note that as each unique image has five or more (say, ```n```) reference captions, each image feature will appear ```n``` times, once in each unique (feature, caption) pair.

In [ ]:
class COCO_Subset(Dataset):
    """ COCO subset custom dataset, compatible with torch.utils.data.DataLoader. """
    
    def __init__(self, df, features, vocab):
        """
        Args:
            df: (dataframe or some other data structure/s you may prefer to use)
            features: image features
            vocab: vocabulary wrapper
           
        """
        
        # TO COMPLETE

    
    def __getitem__(self, index):
        """ Returns one data tuple (feature [1, 2048], target caption of word IDs [1, 47], and integer true caption length) """   
        
       # TO COMPLETE
    
    
    def __len__(self):
        return len(self.features)
    

Load one batch of the training set and print out the shape of each returned Tensor.

In [ ]:
# train_iter = iter(train_loader)
# features, captions, lengths = train_iter.next()
# print(features.shape)
# print(captions.shape)
# print(lengths.shape)

## 3 Train DecoderRNN [15 marks]

### 3.1 Define the encoder model

Read through the ```DecoderRNN``` model below. First, complete the decoder by adding an ```rnn``` layer to the decoder where indicated, using [the PyTorch API as reference](https://pytorch.org/docs/stable/nn.html#rnn).

Keep all the default parameters except for ```batch_first```, which you may set to True.

In particular, understand the meaning of ```pack_padded_sequence()``` as used in ```forward()```. Refer to the [PyTorch ```pack_padded_sequence()``` documentation](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html).


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, embed_size=256, hidden_size=512, num_layers=1, max_seq_length=47):
        """Set the hyper-parameters and build the layers."""
        super(DecoderRNN, self).__init__()
        # we want a specific output size, which is the size of our embedding, so
        # we feed our extracted features from the last fc layer (dimensions 1 x 2048)
        # into a Linear layer to resize
        self.resize = nn.Linear(2048, embed_size)
        
        # batch normalisation helps to speed up training
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)

        # What is an embedding layer?
        self.embed = nn.Embedding(vocab_size, embed_size)

        # TO COMPLETE
        # self.rnn = 

        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_seq_length = max_seq_length
        

    def forward(self, features, captions, lengths):
        """Decode image feature vectors and generates captions."""
        embeddings = self.embed(captions)
        im_features = self.resize(features)
        im_features = self.bn(im_features)
        
        embeddings = torch.cat((im_features.unsqueeze(1), embeddings), 1)
    
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True, enforce_sorted=False) 
        # pack_padded_sequence returns a PackedSequence object, which contains two items: 
        # the packed data (data cut off at its true length and flattened into one list), and 
        # the batch_sizes, or the number of elements at each sequence step in the batch.
        # For instance, given data [a, b, c] and [x] the PackedSequence would contain data 
        # [a, x, b, c] with batch_sizes=[2,1,1].
        
        hiddens, _ = self.rnn(packed)
        outputs = self.linear(hiddens.data) # hiddens[0]
        return outputs
    
    
    def sample(self, features, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []

        inputs = self.bn(self.resize(features)).unsqueeze(1)
        for i in range(self.max_seq_length):
            hiddens, states = self.rnn(inputs, states)  # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))   # outputs:  (batch_size, vocab_size)
            _, predicted = outputs.max(1)               # predicted: (batch_size)
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)              # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)       # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids
    

In [ ]:
# instantiate decoder
decoder = DecoderRNN(len(vocab)).to(device)

### 3.2 Train the decoder

Train the decoder by passing the features, reference captions, and targets to the decoder, then computing loss based on the outputs and the targets. Note that when passing the targets and model outputs to the loss function, the targets will also need to be formatted using ```pack_padded_sequence()```.

We recommend a batch size of around 64 (though feel free to adjust as necessary for your hardware).

**We strongly recommend saving a checkpoint of your trained model after training so you don't need to re-train multiple times.**

Display a graph of training and validation loss over epochs to justify your stopping point.

## 4 Generate predictions on test data [8 marks]

Display 5 sample test images containing different objects, along with your model’s generated captions and all the reference captions for each.

> Remember that everything **displayed** in the submitted notebook and .html file will be marked, so be sure to run all relevant cells.

## 5 Caption evaluation using BLEU score [10 marks]

There are different methods for measuring the performance of image to text models. We will evaluate our model by measuring the text similarity between the generated caption and the reference captions, using two commonly used methods. Ther first method is known as *Bilingual Evaluation Understudy (BLEU)*.

###  5.1 BLEU score


One common way of comparing a generated text to a reference text is using BLEU. This article gives a good intuition to how the BLEU score is computed: https://machinelearningmastery.com/calculate-bleu-score-for-text-python/, and you may find an implementation online to use. One option is the NLTK implementation `nltk.translate.bleu_score` here: https://www.nltk.org/api/nltk.translate.bleu_score.html


> **Tip:** BLEU scores can be weighted by ith-gram. Check that your scores make sense; and feel free to use a weighting that best matches the data. We will not be looking for specific score ranges; rather we will check that the scores are reasonable and meaningful given the captions.

Write the code to evaluate the trained model on the complete test set and calculate the BLEU score using the predictions, compared against all five references captions. 

Display a histogram of the distribution of scores over the test set.

In [ ]:
# TO COMPLETE

### 5.2 BLEU score examples

Find one sample with high BLEU score and one with a low score, and display the model's predicted sentences, the BLEU scores, and the 5 reference captions.

In [ ]:
# TO COMPLETE

## 6 Caption evaluation using cosine similarity [12 marks]

###  6.1 Cosine similarity

The cosine similarity measures the cosine of the angle between two vectors in n-dimensional space. The smaller the angle, the greater the similarity.

To use the cosine similarity to measure the similarity between the generated caption and the reference captions: 

* Find the embedding vector of each word in the caption 
* Compute the average vector for each caption 
* Compute the cosine similarity score between the average vector of the generated caption and average vector of each reference caption
* Compute the average of these scores 

Calculate the cosine similarity using the model's predictions over the whole test set. 

Display a histogram of the distribution of scores over the test set.

In [ ]:
# TO COMPLETE

#### 6.2 Cosine similarity examples 

Find one sample with high cosine similarity score and one with a low score, and display the model's predicted sentences, the cosine similarity scores, and the 5 reference captions.

In [ ]:
# TO COMPLETE

## 7 Comparing BLEU and Cosine similarity [16 marks]

### 7.1 Test set distribution of scores

Compare the model’s performance on the test set evaluated using BLEU and cosine similarity and discuss some weaknesses and strengths of each method (explain in words, in a text box below). 

Please note, to compare the average test scores, you need to rescale the Cosine similarity scores [-1 to 1] to match the range of BLEU method [0.0 - 1.0].

In [ ]:
# TO COMPLETE

 ### 7.2 Analysis of individual examples
 
Find and display one example where both methods give similar scores and another example where they do not and discuss. Include both scores, predicted captions, and reference captions.

In [ ]:
# TO COMPLETE

### Overall quality [5 marks]

See the top of the notebook for submission instructions.